## Downloading and viewing the current HWRF data

In [ ]:
from ipywidgets import *

import numpy as np
import pygrib
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.basemap import Basemap, shiftgrid

import urllib2, urllib

In [ ]:
%matplotlib notebook

In [ ]:
# evaluate path based on date
today=datetime.datetime.today()
date=datetime.datetime.strftime(today,'%Y%m%d')

In [ ]:
#We have to options for the hour 
hh=['00','12']

In [ ]:
try:
 url='http://nomads.ncep.noaa.gov/pub/data/nccf/com/hur/prod/hwrf.{}/'.format(date+hh[1])
 response=urllib2.urlopen(url)
except:
 url='http://nomads.ncep.noaa.gov/pub/data/nccf/com/hur/prod/hwrf.{}/'.format(date+hh[0])
 response=urllib2.urlopen(url)

print url

In [ ]:
ls=response.readlines()

In [ ]:
lp=[elem.strip().split('href=')  for elem in ls]
n=np.size(lp)

In [ ]:
cname=[]
for i in range(n):
     try:
        cc=lp[i][1]
        cname.append(cc.split('"')[1])
     except:
        pass

In [ ]:
cname

In [ ]:
grd3=[x for x in cname if 'hwrfprs.d3.' in x]

In [ ]:
grd3

In [ ]:
urllib.urlretrieve(url+grd3[0], filename='tmp/'+grd3[0])

In [ ]:
data=pygrib.open('tmp/'+grd3[0])

In [ ]:
data.messages

In [ ]:
for g in data:
    print g

In [ ]:
slp = data[1]
u10 = data[706]
v10 = data[707]

In [ ]:
latitudes=u10.latlons()[0][:,0]
longitudes=u10.latlons()[1][0,:]

In [ ]:
u=u10.values
v=v10.values
slp=slp.values/100. # from Pa to mb
wind=np.sqrt(u**2+v**2)

In [ ]:
def viewdata(var,unit,title):
    
    lons, lats = np.meshgrid(longitudes,latitudes)

    # make orthographic basemap.
#    m = Basemap(resolution='c',projection='ortho',lat_0=latc,lon_0=lonc)
#   m = Basemap(projection='cea',llcrnrlat=-90,urcrnrlat=90,\
#            llcrnrlon=-180,urcrnrlon=180,resolution='c')
    m = Basemap(projection='cyl',llcrnrlat=latitudes.min(),urcrnrlat=latitudes.max(),\
              llcrnrlon=longitudes.min(),urcrnrlon=longitudes.max(),resolution='i')
    # create figure, add axes
    fig1 = plt.figure(figsize=(12,10))
    ax = fig1.add_axes([0.1,0.1,0.8,0.8])
    # set desired contour levels.
    clevs = np.linspace(var.min(),var.max(), 10,endpoint=True)

    # compute native x,y coordinates of grid.
    x, y = m(lons, lats)
    # define parallels and meridians to draw.
    parallels = np.arange(-90.,90,20.)
    meridians = np.arange(0.,360.,20.)
    # plot SLP contours.
 #   slpg,newlons = shiftgrid(180.,slp,longitudes,start=False)
 #   slpd,xx,yy = \
 #   m.transform_scalar(slpg,newlons,latitudes,181,181,returnxy=True,masked=True)
    CS1 = m.contourf(x,y,var,clevs,cmap=plt.cm.RdBu_r,animated=True)
    CS2 = m.contour(x,y,var,clevs,linewidths=0.5,colors='k',animated=True)

   # draw coastlines, parallels, meridians.
    m.drawcoastlines(linewidth=1.5)
    m.drawparallels(parallels)
    m.drawmeridians(meridians)
    # add colorbar
    cb = m.colorbar(CS1,"bottom", size="5%", pad="2%", ticks=clevs)
    cb.set_label(unit)
    # set plot title
    ax.set_title('HWRF '+title)
    plt.show()

In [ ]:
viewdata(slp,'mb','pressure')

In [ ]:
viewdata(wind,'m/s','wind')